In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np
import pandas as pd

In [21]:
STORM_NUMBER = 4
TITLE = "1-2 January 2022"
SUBTITLE = "8 AM 2 January 2022"
sts = utc(2022, 1, 1, 18)
ets = utc(2022, 1, 30, 20, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 60, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [22]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2021-2022 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'].values,
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [32]:
cull = ['GRTM7', 352, 461, 290, 'SGYI4', 117, 137, 'IA-WS-7', 114, 119, 146, 147, 115, 48, 'DSXI4', 36, 40,
       112, 'DVNI4']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [

]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=6, upper=400)
res = mp.postprocess(filename='220102.png')
mp.close()

     state  wfo   val       lon      lat                               geo  \
0       AK  AJK  30.0 -130.0300  55.9200  POINT (-1819334.227 1574082.671)   
1       AK  AJK  24.0 -139.7300  59.5500  POINT (-2125633.723 2191635.469)   
2       CO  GJT  22.0 -107.1100  38.8900   POINT (-615310.861 -652769.985)   
3       AK  AJK  17.3 -134.4500  58.3000  POINT (-1938907.052 1925615.468)   
4       CO  GJT  16.0 -107.7000  37.7400   POINT (-677258.767 -775529.859)   
...    ...  ...   ...       ...      ...                               ...   
2445    WI  GRB   0.0  -87.8119  45.4825     POINT (945709.523 125196.948)   
2446    WI  GRB   0.0  -88.9056  45.6719     POINT (858648.796 133851.846)   
2447    WI  GRB   0.0  -88.4377  44.2769     POINT (916646.214 -14988.856)   
2448    WI  GRB   0.0  -89.7267  45.8994     POINT (792312.235 150567.651)   
2449    WI  GRB   0.0  -89.1133  44.6398      POINT (858124.569 17678.059)   

      used_for_analysis  nwsli  plotme source  
0              